In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff


In [2]:
yandex_russia = pd.read_csv('yandex_russia.csv')

yandex_russia.rename(columns = {'Дата':'Date','Регион':'Region','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)


for i in range(yandex_russia.shape[0]):
    a= yandex_russia.Date[i].split('.')
    yandex_russia.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))
    
   
yandex_russia.tail(10)

,Date,Region,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered
13953,2020-08-22,Ульяновская обл.,11302,8438,87,7,87,145
13954,2020-08-22,Хабаровский край,9443,7367,83,0,68,69
13955,2020-08-22,Хакасия,3253,3004,39,0,19,16
13956,2020-08-22,ХМАО – Югра,18999,17327,156,1,83,170
13957,2020-08-22,Челябинская обл.,13090,8817,103,1,71,135
13958,2020-08-22,Чечня,2222,1511,38,0,7,4
13959,2020-08-22,Чувашия,7481,6169,68,0,33,55
13960,2020-08-22,Чукотский АО,167,160,2,0,0,0
13961,2020-08-22,Ямало-Ненецкий АО,12957,12420,87,2,63,138
13962,2020-08-22,Ярославская обл.,6885,6133,32,0,38,39


In [3]:
df3_temp = yandex_russia.groupby('Date').sum()
df3_temp['Region']='Россия'

y5= pd.concat([yandex_russia,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_russia = y5
yandex_russia.tail(10)

,Date,Region,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered
164,2020-08-13,Россия,907758,716396,15384,124,5057,6098
165,2020-08-14,Россия,912823,722964,15498,114,5065,6568
166,2020-08-15,Россия,917884,729411,15617,119,5061,6447
167,2020-08-16,Россия,922853,732968,15685,68,4969,3557
168,2020-08-17,Россия,927745,736101,15740,55,4892,3133
169,2020-08-18,Россия,932493,742628,15872,132,4748,6527
170,2020-08-19,Россия,937321,749423,15989,117,4828,6795
171,2020-08-20,Россия,942106,755513,16099,110,4785,6090
172,2020-08-21,Россия,946976,761330,16189,90,4870,5817
173,2020-08-22,Россия,951897,767477,16310,121,4921,6147


In [4]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_russia.tail(1).Date)[0]

print(yandex_russia.iloc[-1,0])
print('Выбран день: ',myday)

2020-08-22
Выбран день:  2020-08-22


In [5]:
X = pd.DataFrame()
regions_dict = {}
yandex_russia['Rt'] = 1
yandex_russia['Rt2'] = 1
yandex_russia['Rate_conf'] = 0
yandex_russia['Rate_ill'] = 0

temp_df = yandex_russia.groupby(['Region','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Region'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    regions_dict[i]=j
    
print(regions_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Адыгея'), (1, 'Алтай'), (2, 'Алтайский край'), (3, 'Амурская обл.'), (4, 'Архангельская обл.'), (5, 'Астраханская обл.'), (6, 'Башкортостан'), (7, 'Белгородская обл.'), (8, 'Брянская обл.'), (9, 'Бурятия'), (10, 'Владимирская обл.'), (11, 'Волгоградская обл.'), (12, 'Вологодская обл.'), (13, 'Воронежская обл.'), (14, 'Дагестан'), (15, 'Еврейская АО'), (16, 'Забайкальский край'), (17, 'Ивановская обл.'), (18, 'Ингушетия'), (19, 'Иркутская обл.'), (20, 'Кабардино-Балкария'), (21, 'Калининградская обл.'), (22, 'Калмыкия'), (23, 'Калужская обл.'), (24, 'Камчатский край'), (25, 'Карачаево-Черкессия'), (26, 'Карелия'), (27, 'Кемеровская обл.'), (28, 'Кировская обл.'), (29, 'Коми'), (30, 'Костромская обл.'), (31, 'Краснодарский край'), (32, 'Красноярский край'), (33, 'Крым'), (34, 'Курганская обл.'), (35, 'Курская обл.'), (36, 'Ленинградская обл.'), (37, 'Липецкая обл.'), (38, 'Магаданская обл.'), (39, 'Марий Эл'), (40, 'Мордовия'), (41, 'Москва'), (42, 'Московская обл.'), (4

,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
14132,2020-08-18,6732,5999,32,0,37,51,0.994,1.024,0,0,Ярославская обл.,701,38.571429,38.714286,665.857143,648.214286,-3,-14
14133,2020-08-19,6772,6033,32,0,40,34,1.013,1.024,0,0,Ярославская обл.,707,38.857143,38.785714,675.571429,649.500000,3,6
14134,2020-08-20,6810,6059,32,0,38,26,1.000,1.023,0,0,Ярославская обл.,719,38.285714,38.714286,686.428571,654.000000,-2,12
14135,2020-08-21,6847,6094,32,0,37,35,0.993,1.023,0,0,Ярославская обл.,721,38.428571,38.428571,697.571429,662.571429,-1,2
14136,2020-08-22,6885,6133,32,0,38,39,0.994,1.023,0,0,Ярославская обл.,720,38.428571,38.500000,708.714286,673.000000,1,-1


In [6]:
mask = X.Region == 'Россия'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
rus_date = list(temp_df.index)
rus_conf = list(temp_df.Confirmed)
rus_ill = list(temp_df.Remaining_ill)
rus_dict_conf = dict(zip(rus_date,rus_conf))
rus_dict_ill = dict(zip(rus_date,rus_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/rus_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/rus_dict_ill.get(X.loc[i,'Date'])


Confirmed  Remaining_ill
Date                                
2020-08-21     946976         169457
2020-08-22     951897         168110
14137
(14137, 19)
2020-03-12


In [7]:
mask = (X.Date>=datetime.date(2020,6,13))&(X.Region.isin(['Москва','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Region','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Region,Rate_conf,Rate_ill
9197,2020-08-03,856264,Россия,100.0,100.0
9198,2020-08-04,861423,Россия,100.0,100.0
9199,2020-08-05,866627,Россия,100.0,100.0
9200,2020-08-06,871894,Россия,100.0,100.0
9201,2020-08-07,877135,Россия,100.0,100.0
9202,2020-08-08,882347,Россия,100.0,100.0
9203,2020-08-09,887536,Россия,100.0,100.0
9204,2020-08-10,892654,Россия,100.0,100.0
9205,2020-08-11,897599,Россия,100.0,100.0
9206,2020-08-12,902701,Россия,100.0,100.0


In [8]:
# russia_regions = pd.read_csv('regions-info.csv')

# russia_regions[(russia_regions.Region =='Московская область')|(russia_regions.Region =='Москва')|
#               (russia_regions.Region =='Санкт-Петербург')].tail()

In [9]:
X.to_csv('X.csv',index=False)


In [10]:
covid_rates = pd.DataFrame()
mask = (X.Date==myday)&(X.Region!='Россия')
today_cases = X[mask]
#covid_rates.reset_index(drop=True)

covid_rates['По количеству текущих больных'] = today_cases.sort_values(by=['Remaining_ill'],ascending = False).Region.reset_index(drop=True)
covid_rates['По количеству новых'] = today_cases.sort_values(by=['Day_confirmed'],ascending = False).Region.reset_index(drop=True)
covid_rates['По умершим за день'] = today_cases.sort_values(by=['Day_deaths'],ascending = False).Region.reset_index(drop=True)
covid_rates['По выздоровевшим за день'] = today_cases.sort_values(by=['Day_recovered'],ascending = False).Region.reset_index(drop=True)

fig =  ff.create_table(covid_rates.head(15))
fig.update_layout(
    
                  title_text ='Таблица рейтингов заболеваемости COVID-19 по регионам',
                  margin = {'t':100, 'b':100},
                  title_x = 0.5,
                  title_y= 0.95,
                  title_xanchor = "center",
                  title_yanchor = "top", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
)
fig.show()




In [11]:
region_number = 5
regions_towatch = list(covid_rates['По количеству текущих больных'].head(region_number))
regions_towatch.append("Россия")
regions_towatch

['Москва',
 'Московская обл.',
 'Санкт-Петербург',
 'Свердловская обл.',
 'Челябинская обл.',
 'Россия']

In [12]:
mask = (X.Date>datetime.date(2020,5,1))&(X.Region.isin(regions_towatch))
#mask = (X.Region.isin(regions_towatch))

fig = px.line(X[mask], y="Day_confirmed", x="Date",
                             color="Region", line_group="Region",
              line_shape="spline", render_mode="notebook",labels={'Region':'Регион:'},
              color_discrete_map={
                "Москва": "red",
                "Московская обл.": "magenta",
                "Санкт-Петербург": "goldenrod",
                "Нижегородская обл.": "darkblue",
                "Свердловская обл.": "green",
                 'Россия': 'aquamarine'},
             title="График количества новых больных по 5 регионам с наибольшим количеством заболеваний")

fig.update_layout(
    annotations=[
            dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper")],
           
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ''
)

fig.show()






In [13]:
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region.isin(regions_towatch))

colors_dict = {"Москва": "red",
                "Московская обл.": "magenta",
                "Санкт-Петербург": "goldenrod",
                "Нижегородская обл.": "darkblue",
                "Свердловская обл.": "green",
                 'Россия':'aquamarine'}
fig = go.Figure()
for i in regions_towatch:
    mask = X['Region'] == i
        
    fig.add_trace(
        
        go.Bar(
        name = f'{i}',
        x=X[mask].Date,
        y=X[mask].Rt,
        text=round(X[mask].Rt,3),
        textposition='inside',
        marker_color= colors_dict.get(i),
        
        showlegend=True))




                  
fig.update_layout(
    annotations=[
        dict(
            x='2020-04-13',
            y=1.717,
            xref="x",
            yref="y",
            text="Введение<br>пропускной<br>системы в Москве",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-60),
        dict(
            x='2020-06-09',
            y=0.942,
            xref="x",
            yref="y",
            text="Отмена<br>пропускной<br>системы в Москве",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-60),
            dict(x=0.5,y=-0.15,
            showarrow=False,
            text="Данные : Яндекс <br> формула расчета из методических рекомендациях Роспотребнадзора.",
            xref="paper",
            yref="paper")],
                  title = "Коэффициент распространения (Rt) по 5 регионам ,<br> с максимальным количеством новых заболеваний ",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.9,
                  width = 990, height = 600,template = 'gridon',
    
    
                 # xaxis_title='Коэффициент рассчитан, исходя из данных оперативного штаба и <br>' + 
                               #' формулы расчета в методических рекомендациях Роспотребнадзора. ',
    yaxis_title = ' '
)


fig.show()


In [14]:
print('Все регионы по которым есть статистика:')
print('*'*40)
print(regions_dict.items())
print('*'*40)
print('Рассматриваем регионы: ')
for i in regions_dict.keys():
    if regions_dict.get(i) in regions_towatch:
        print(i,regions_dict.get(i))


my_region = regions_dict[41]
print('Выбрали регион: ',my_region)


Все регионы по которым есть статистика:
****************************************
dict_items([(0, 'Адыгея'), (1, 'Алтай'), (2, 'Алтайский край'), (3, 'Амурская обл.'), (4, 'Архангельская обл.'), (5, 'Астраханская обл.'), (6, 'Башкортостан'), (7, 'Белгородская обл.'), (8, 'Брянская обл.'), (9, 'Бурятия'), (10, 'Владимирская обл.'), (11, 'Волгоградская обл.'), (12, 'Вологодская обл.'), (13, 'Воронежская обл.'), (14, 'Дагестан'), (15, 'Еврейская АО'), (16, 'Забайкальский край'), (17, 'Ивановская обл.'), (18, 'Ингушетия'), (19, 'Иркутская обл.'), (20, 'Кабардино-Балкария'), (21, 'Калининградская обл.'), (22, 'Калмыкия'), (23, 'Калужская обл.'), (24, 'Камчатский край'), (25, 'Карачаево-Черкессия'), (26, 'Карелия'), (27, 'Кемеровская обл.'), (28, 'Кировская обл.'), (29, 'Коми'), (30, 'Костромская обл.'), (31, 'Краснодарский край'), (32, 'Красноярский край'), (33, 'Крым'), (34, 'Курганская обл.'), (35, 'Курская обл.'), (36, 'Ленинградская обл.'), (37, 'Липецкая обл.'), (38, 'Магаданская обл.')

In [15]:
mask = X.Region ==my_region
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region.isin(regions_towatch))
#mask = (X.Date>datetime.date(2020,5,1)&(X.Region==my_region)

fig = px.bar(X[mask], x='Date', 
             y="Day_confirmed",color ='Day_confirmed',
             title=f'{my_region}: '+'график количества новых заболевших COVID-19 по датам',
              color_continuous_scale= 'jet',text = 'Day_confirmed',
             labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)
fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask]['Date'],
        y=X[mask]['MA7_dayconfirmed'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=1355,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=1572,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-90),
        dict(
            x=0.5,
            y=-0.15,
            showarrow=False,
            text="Данные: Яндекс",
            xref="paper",
            yref="paper"),
        
                ],
    
    
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.15,legend_y = 0.95,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='',yaxis_title = ' '
)


fig.show()




In [16]:
mask = X.Region ==my_region
#mask = (X.Date>datetime.date(2020,5,1))&(X.Region==my_region)
#Y = X[mask]

#Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
#Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = X[mask].Date,y=X[mask].Change_dayconf,text = X[mask]['Change_dayconf'],textposition = 'inside',
                
                
                 marker_color = X[mask]['Change_dayconf'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}}
                    )   
                )
fig.update_layout(
   annotations=[
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_region}: динамика новых заболевших COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Данные: Яндекс',
    yaxis_title = ' '
)

fig.show()






In [17]:
mask = X.Region ==my_region
#mask = (X.Date>'2020-05-01')&(X.Region.isin(regions_towatch))

fig = px.bar(X[mask], x='Date', y="Remaining_ill",color ='Remaining_ill',
             title=f'{my_region}: '+'график количества текущих больных COVID-19 по датам',
              color_continuous_scale= 'tempo',text = 'Remaining_ill',
            labels = {'Day_confirmed' : "Новые заболевшие",'Remaining_ill':'Текущие больные'})
fig.add_trace(
    go.Scatter(
        name='14-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA14_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="green"),
        showlegend=True)
)

fig.add_trace(
    go.Scatter(
        name = '7-дневная скользящая средняя',
        x=X[mask].Date,
        y=X[mask]['MA7_remaining_ill'],
        mode="lines",
        
        line=go.scatter.Line(color="red"),
        showlegend=True)
)
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=85824,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                                '<br>Данные: Яндекс',
    yaxis_title = ' '
)


fig.show()





In [18]:
mask = X.Region ==my_region
#mask = (X.Date>'2020-05-01')&(X.Region==my_region)
Y = X[mask]

#Y['base'] = Y['Change_dayconf']*Y['Change_dayconf'].apply(lambda d:0 if d>0 else 1)
Y['Date']= pd.to_datetime(Y.Date)
fig = go.Figure()
fig.add_trace(
              go.Bar(x = Y.Date,y=Y.Change_remill,text = Y['Change_remill'],textposition = 'inside',
                
                #marker_color=Y['Change_remill'].apply(lambda d:'crimson' if d>0 else 'green'),
                marker_color = Y['Change_remill'],marker_colorscale = 'Temps',marker_colorbar ={'tickmode':'auto',
                                    'title':{'text':'Изменение','side':'top'}})     
                )
fig.update_layout(
    annotations=[
#         dict(
#             x='2020-04-13',
#             y=10594,
#             xref="x",
#             yref="y",
#             text="Введение<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=0,
#             ay=-60),
#         dict(
#             x='2020-06-09',
#             y=-3877,
#             xref="x",
#             yref="y",
#             text="Отмена<br>пропускной<br>системы в Москве",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
#         dict(
#             x='2020-05-20',
#             y=-2816,
#             xref="x",
#             yref="y",
#             text="20 мая",
#             showarrow=True,
#             arrowhead=2,
#             ax=-20,
#             ay=40),
        
        dict(x=0.5,y=-0.15,
            showarrow=False,
            text="",
            xref="paper",
            yref="paper")],
                  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                  legend_x = 0.05,legend_y = 0.98,
                  width = 990, height = 600,template = 'gridon',
    
    
                  xaxis_title='Текущие больные = количество выявленных - количество выздоровевших - количество умерших'+
                               '<br>Данные : Яндекс',
    yaxis_title = ' '
)

fig.show()


In [19]:
mask = (X.Date>=datetime.date(2020,7,30))&(X.Region.isin(regions_towatch))

def my_round(k):
    return round(k,3)

# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Region','Date']).Rt.sum().apply(my_round).unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Region',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.head(10)
print(regions_towatch)

[[1.051 1.018 1.013 1.002 1.009 1.007 1.    1.01  1.002 1.004 1.005 1.002
  1.007 1.007 1.007 1.001 0.999 0.999 0.997 1.001 0.999 1.    1.    0.996]
 [0.941 0.97  1.041 1.099 1.128 1.126 1.103 1.07  1.039 1.021 0.98  0.967
  0.964 0.95  0.968 0.982 0.987 1.005 1.015 0.992 0.99  0.992 0.981 1.025]
 [0.945 0.947 0.967 0.982 0.989 0.981 0.966 0.961 0.959 0.976 0.987 0.987
  0.98  0.973 0.967 0.972 0.991 0.99  0.988 0.975 0.958 0.955 0.962 0.986]
 [0.928 0.925 0.932 0.949 0.964 0.982 0.985 0.981 0.98  0.977 0.991 0.994
  0.998 1.013 1.006 1.013 1.021 1.014 1.013 1.006 0.998 0.991 1.026 1.07 ]
 [0.909 0.864 0.892 0.893 0.898 0.909 0.892 0.898 0.9   0.927 0.956 0.972
  0.977 0.967 0.932 0.903 0.904 0.894 0.892 0.919 0.916 0.923 0.927 0.888]
 [0.98  0.962 0.929 0.896 0.893 0.889 0.9   0.895 0.878 0.861 0.854 0.867
  0.897 0.943 0.939 0.955 0.94  0.911 0.894 0.864 0.872 0.871 0.928 0.983]]
['Москва', 'Московская обл.', 'Санкт-Петербург', 'Свердловская обл.', 'Челябинская обл.', 'Россия']


In [20]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Region)
print(list(x1))
print('-'*40)
print(x2)

[datetime.date(2020, 7, 30), datetime.date(2020, 7, 31), datetime.date(2020, 8, 1), datetime.date(2020, 8, 2), datetime.date(2020, 8, 3), datetime.date(2020, 8, 4), datetime.date(2020, 8, 5), datetime.date(2020, 8, 6), datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9), datetime.date(2020, 8, 10), datetime.date(2020, 8, 11), datetime.date(2020, 8, 12), datetime.date(2020, 8, 13), datetime.date(2020, 8, 14), datetime.date(2020, 8, 15), datetime.date(2020, 8, 16), datetime.date(2020, 8, 17), datetime.date(2020, 8, 18), datetime.date(2020, 8, 19), datetime.date(2020, 8, 20), datetime.date(2020, 8, 21), datetime.date(2020, 8, 22)]
----------------------------------------
['30.7.2020', '31.7.2020', '1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020', '16.8.2020', '17.8.2020', '18.8.2020', '19.8.2020', '20.8.2020', '21.8.2020', '22.

In [21]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()


In [22]:
mask = (X.Date>=datetime.date(2020,7,27))&(X.Region.isin(regions_towatch))


# #mask = (X.Date>='2020-06-04')

Z = X[mask].groupby(['Region','Date']).Rt2.sum().unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Region',axis = 1).values

# my_map= {'Москва':'2.Москва','Московская обл.':'3.Московская обл.','Нижегородская обл.':'5.Нижегородская обл.',
#                     'Россия':'1.Россия', 'Санкт-Петербург':'4.Санкт-Петербург'}
# Z.Region = Z.Region.map(my_map)

print(d)
Z.head()

[[1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011
  1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011 1.011
  1.011 1.011 1.011]
 [1.009 1.009 1.009 1.009 1.009 1.009 1.009 1.009 1.01  1.01  1.01  1.01
  1.01  1.01  1.01  1.01  1.01  1.009 1.009 1.009 1.009 1.009 1.009 1.009
  1.009 1.009 1.009]
 [1.03  1.029 1.029 1.028 1.027 1.027 1.027 1.026 1.026 1.026 1.025 1.025
  1.025 1.024 1.024 1.024 1.024 1.023 1.023 1.023 1.023 1.022 1.022 1.022
  1.021 1.021 1.021]
 [1.024 1.024 1.023 1.023 1.022 1.022 1.021 1.021 1.021 1.02  1.02  1.02
  1.02  1.02  1.02  1.019 1.019 1.019 1.019 1.019 1.019 1.019 1.019 1.019
  1.019 1.019 1.019]
 [1.053 1.053 1.051 1.049 1.047 1.045 1.043 1.042 1.04  1.039 1.037 1.036
  1.035 1.034 1.034 1.033 1.033 1.032 1.031 1.03  1.029 1.028 1.027 1.026
  1.026 1.025 1.024]
 [1.051 1.05  1.049 1.048 1.047 1.046 1.044 1.043 1.041 1.039 1.038 1.036
  1.035 1.033 1.032 1.03  1.03  1.029 1.029 1.028 1.027 1.026 1.025 1.024
  1.023 1

Date,Region,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,...,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22
0,Москва,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,...,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011
1,Московская обл.,1.009,1.009,1.009,1.009,1.009,1.009,1.009,1.009,1.010,...,1.009,1.009,1.009,1.009,1.009,1.009,1.009,1.009,1.009,1.009
2,Россия,1.030,1.029,1.029,1.028,1.027,1.027,1.027,1.026,1.026,...,1.023,1.023,1.023,1.023,1.022,1.022,1.022,1.021,1.021,1.021
3,Санкт-Петербург,1.024,1.024,1.023,1.023,1.022,1.022,1.021,1.021,1.021,...,1.019,1.019,1.019,1.019,1.019,1.019,1.019,1.019,1.019,1.019
4,Свердловская обл.,1.053,1.053,1.051,1.049,1.047,1.045,1.043,1.042,1.040,...,1.032,1.031,1.030,1.029,1.028,1.027,1.026,1.026,1.025,1.024


In [23]:
x1=list(Z.columns[1:])
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
y=list(Z.Region)
print(list(x1))
print(x2)

[datetime.date(2020, 7, 27), datetime.date(2020, 7, 28), datetime.date(2020, 7, 29), datetime.date(2020, 7, 30), datetime.date(2020, 7, 31), datetime.date(2020, 8, 1), datetime.date(2020, 8, 2), datetime.date(2020, 8, 3), datetime.date(2020, 8, 4), datetime.date(2020, 8, 5), datetime.date(2020, 8, 6), datetime.date(2020, 8, 7), datetime.date(2020, 8, 8), datetime.date(2020, 8, 9), datetime.date(2020, 8, 10), datetime.date(2020, 8, 11), datetime.date(2020, 8, 12), datetime.date(2020, 8, 13), datetime.date(2020, 8, 14), datetime.date(2020, 8, 15), datetime.date(2020, 8, 16), datetime.date(2020, 8, 17), datetime.date(2020, 8, 18), datetime.date(2020, 8, 19), datetime.date(2020, 8, 20), datetime.date(2020, 8, 21), datetime.date(2020, 8, 22)]
['27.7.2020', '28.7.2020', '29.7.2020', '30.7.2020', '31.7.2020', '1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020', 

In [24]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,    
    
                  xaxis_title='Динамика коэффициента распространения Rt2 по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()


In [25]:
mask = (X.Date>=datetime.date(2020,7,27))&(X.Region.isin(regions_towatch))

Z = X[mask].groupby(['Region','Date'])['Day_confirmed'].sum().unstack().reset_index()
Z.dropna(inplace=True)
d = Z.drop('Region',axis = 1).values
Z.head()

Date,Region,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,...,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22
0,Москва,694,674,671,678,695,690,664,693,691,...,692,688,695,688,690,693,691,688,690,687
1,Московская обл.,139,126,137,144,150,149,157,160,161,...,148,156,153,154,153,145,151,157,151,161
2,Россия,5635,5395,5475,5509,5482,5462,5427,5394,5159,...,5057,5065,5061,4969,4892,4748,4828,4785,4870,4921
3,Санкт-Петербург,172,168,165,162,159,161,163,160,158,...,160,159,162,163,159,157,162,160,181,183
4,Свердловская обл.,252,209,223,215,210,202,199,196,182,...,162,154,158,150,141,149,147,139,124,121


In [26]:



x1=list(Z.columns[1:])
y=list(Z.Region)
x2=[]
for i in x1:
    x2.append(str(i.day)+'.'+str(i.month)+"."+str(i.year))
    
print(x2)


['27.7.2020', '28.7.2020', '29.7.2020', '30.7.2020', '31.7.2020', '1.8.2020', '2.8.2020', '3.8.2020', '4.8.2020', '5.8.2020', '6.8.2020', '7.8.2020', '8.8.2020', '9.8.2020', '10.8.2020', '11.8.2020', '12.8.2020', '13.8.2020', '14.8.2020', '15.8.2020', '16.8.2020', '17.8.2020', '18.8.2020', '19.8.2020', '20.8.2020', '21.8.2020', '22.8.2020']


In [27]:
fig = ff.create_annotated_heatmap(d, x=x2, y=y, annotation_text=d,
                                  colorscale='Temps')

fig.update_layout(
                  
                  
                  width = 990, height = 600,
    
    
                  xaxis_title='Динамика новых заболевших по регионам',
               #   yaxis_title = 'Регион '
)

fig.show()

In [28]:
mask = (X.Date==myday)&(X.Region!='Россия')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_conf'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_conf']<1:
        df2.loc[i,'Region']='Регионы менее 1%'
df.head(15)

85


,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
6899,2020-08-22,256513,203821,4720,10,687,1094,0.996,1.011,26.947558,28.536078,Москва,47972,689.571429,690.571429,49372.571429,50837.571429,-3,-417
7073,2020-08-22,66853,50348,1161,1,161,154,1.025,1.009,7.023134,9.127357,Московская обл.,15344,153.142857,152.642857,15308.428571,15306.785714,10,6
9877,2020-08-22,35011,25087,2332,18,183,110,1.070,1.019,3.678024,4.516091,Санкт-Петербург,7592,166.428571,162.928571,7403.000000,7157.571429,2,55
7566,2020-08-22,26513,23642,445,2,122,196,1.072,1.020,2.785280,1.443103,Нижегородская обл.,2426,125.428571,124.428571,2589.285714,2712.571429,-5,-76
10533,2020-08-22,24226,19443,403,7,121,60,0.888,1.024,2.545023,2.605437,Свердловская обл.,4380,138.714286,153.071429,4201.571429,3951.928571,-3,54
12829,2020-08-22,18999,17327,156,1,83,170,0.988,1.017,1.995909,0.901790,ХМАО – Югра,1516,81.571429,98.571429,1775.857143,2059.714286,-2,-88
3276,2020-08-22,16086,11959,239,0,105,121,0.932,1.028,1.689889,2.312771,Иркутская обл.,3888,108.857143,113.285714,3954.428571,3987.642857,4,-16
5412,2020-08-22,15862,12893,416,6,114,85,0.979,1.030,1.666357,1.518649,Красноярский край,2553,115.857143,119.428571,2434.142857,2231.928571,-3,23
9380,2020-08-22,15761,13318,306,5,129,159,0.996,1.034,1.655746,1.271191,Ростовская обл.,2137,127.714286,127.214286,2293.285714,2340.500000,1,-35
2292,2020-08-22,13244,11784,104,0,94,97,1.039,1.031,1.391327,0.806615,Воронежская обл.,1356,97.714286,96.214286,1430.285714,1581.857143,-3,-3


In [29]:



fig = px.pie(df2, values='Confirmed', names='Region',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего выявленных случаев заболевания COVID-19 в России: {df2.Confirmed.sum()}<br>'+
            f'(на дату: {myday})'
            ,
            width = 900, height=1000)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  title_x = 0.5,
                  margin = {'t':150, 'b':0},
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()


In [30]:
mask = (X.Date==myday)&(X.Region!='Россия')
#mask = (X.Date==myday)
#Делаем выборку по доле регина в общих случаях выявления заболеваний по России
df=X[mask].sort_values(by=['Rate_ill'],ascending = False)
#Далее заменяем все имена если регион набрал менее 1% в общих случаях по России
df2 = X[mask].reset_index(drop=True)
print(len(df2))

for i in range(len(df2)):
    if df2.loc[i,'Rate_ill']<1:
        df2.loc[i,'Region']='Регионы менее 1%'
df.head(20)

85


,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
6899,2020-08-22,256513,203821,4720,10,687,1094,0.996,1.011,26.947558,28.536078,Москва,47972,689.571429,690.571429,49372.571429,50837.571429,-3,-417
7073,2020-08-22,66853,50348,1161,1,161,154,1.025,1.009,7.023134,9.127357,Московская обл.,15344,153.142857,152.642857,15308.428571,15306.785714,10,6
9877,2020-08-22,35011,25087,2332,18,183,110,1.070,1.019,3.678024,4.516091,Санкт-Петербург,7592,166.428571,162.928571,7403.000000,7157.571429,2,55
10533,2020-08-22,24226,19443,403,7,121,60,0.888,1.024,2.545023,2.605437,Свердловская обл.,4380,138.714286,153.071429,4201.571429,3951.928571,-3,54
13321,2020-08-22,13090,8817,103,1,71,135,0.983,1.022,1.375149,2.480519,Челябинская обл.,4170,71.571429,77.928571,4284.285714,4341.785714,-2,-65
10041,2020-08-22,11494,7415,68,0,95,110,0.972,1.035,1.207484,2.385938,Саратовская обл.,4011,95.714286,95.500000,4069.428571,4012.857143,4,-15
3276,2020-08-22,16086,11959,239,0,105,121,0.932,1.028,1.689889,2.312771,Иркутская обл.,3888,108.857143,113.285714,3954.428571,3987.642857,4,-16
1964,2020-08-22,11190,7599,84,1,85,137,0.909,1.033,1.175547,2.086134,Волгоградская обл.,3507,87.714286,92.714286,3672.714286,3657.142857,3,-53
11189,2020-08-22,10869,7365,212,3,128,176,1.016,1.052,1.141825,1.958242,Ставропольский край,3292,130.428571,123.785714,3250.285714,3058.714286,-4,-51
12665,2020-08-22,11302,8438,87,7,87,145,0.902,1.035,1.187313,1.651895,Ульяновская обл.,2777,93.714286,97.642857,2966.142857,2925.500000,-4,-65


In [31]:


fig = px.pie(df2, values='Remaining_ill', names='Region',color_discrete_sequence=px.colors.sequential.Rainbow,
            title=f'Всего текущих больных COVID-19 в России: {df2.Remaining_ill.sum()}<br>'+
            f'(на дату: {myday})'
             ,
            width = 900, height=1000)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
        
                #  title=f'{my_region}: динамика текущих больных COVID-19 по дням<br>(изменение относительно предыдущего дня)',
                  margin = {'t':150, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "bottom", 
                 # legend_x = 0.05,legend_y = 0.98,
                     
)

fig.show()


In [32]:
fig_r8 = go.Figure(data=[go.Pie(labels=df2.Region, values=df2.Remaining_ill, textinfo='label+percent',
                             insidetextorientation='radial'
                            )])
fig_r8.update_layout(
                title=f'Всего текущих больных COVID-19 в России: {df2.Remaining_ill.sum()}<br>'+
                f'(на дату: {myday})',
                 margin = {'t':120, 'b':0},
                  title_x = 0.5,
                  title_y= 0.9,
                  width = 900, height=900,
                  title_xanchor = "center",
                  title_yanchor = "bottom")
fig_r8.show()

In [33]:
covid_rates = pd.DataFrame()
mask = (X.Date>=datetime.date(2020,8,3))&(X.Date<=datetime.date(2020,8,10))&(X.Region.isin(regions_towatch))&(X.Region!='Россия')
                                      
box_cases1 = X[mask].groupby(['Region','Date'])['Day_confirmed'].sum()
box_cases2 = X[mask].groupby(['Region','Date'])['Day_recovered'].sum()
for i in regions_dict.keys():
    if regions_dict.get(i) in regions_towatch:
        print(i,regions_dict.get(i))

41 Москва
42 Московская обл.
55 Россия
59 Санкт-Петербург
63 Свердловская обл.
80 Челябинская обл.


In [34]:
new_df1=box_cases1.unstack().T
new_df1.tail(20)

Region,Москва,Московская обл.,Санкт-Петербург,Свердловская обл.,Челябинская обл.
Date,,,,,
2020-08-03,693,160,160,196,107
2020-08-04,691,161,158,182,105
2020-08-05,687,162,156,181,103
2020-08-06,684,159,159,183,92
2020-08-07,686,158,157,180,89
2020-08-08,691,161,155,178,87
2020-08-09,689,149,160,184,94
2020-08-10,694,153,157,179,83


In [35]:
color_dict = {
                "Москва": "red",
                "Московская обл.": "magenta",
                "Санкт-Петербург": "goldenrod",
                "Иркутская обл.": "darkblue",
                "Свердловская обл.": "green",
                 'Челябинская обл.': 'indianred'}

cols = list(new_df1.columns)
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по новым заболевшим за последнюю неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Новые заболевшие', yaxis_title='Регионы',width = 990, height = 600)
fig.show()

In [36]:
new_df2=box_cases2.unstack().T
new_df2.tail(20)

Region,Москва,Московская обл.,Санкт-Петербург,Свердловская обл.,Челябинская обл.
Date,,,,,
2020-08-03,842,197,75,107,3
2020-08-04,1456,207,85,96,170
2020-08-05,1329,191,37,92,142
2020-08-06,1254,189,84,95,141
2020-08-07,1353,194,93,90,120
2020-08-08,1278,191,83,84,119
2020-08-09,803,176,33,82,29
2020-08-10,898,169,45,80,78


In [37]:

cols = list(new_df2.columns)
#cols = cols[:-1]

fig = go.Figure()
for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)))

fig.update_layout(title_text="Статистика по выздоровевшим за последнюю неделю",
                  title_x = 0.5,
                  title_y= 0.9,
                  title_xanchor = "center",
                  title_yanchor = "top",
                  xaxis_title='Выздоровевшие', yaxis_title='Регионы',width = 990, height = 600)
fig.show()

In [38]:
from plotly.subplots import make_subplots

# fig = tools.make_subplots(rows = 1, cols = 2, specs = [[{}, {}]],
#                           horizontal_spacing = 0.05)


fig = make_subplots(rows=2, cols=1,specs = [[{}],[{}]],vertical_spacing = 0.03,shared_xaxes=True,
                  subplot_titles=("Box-график распределения по новым заболевшим(в день) за неделю:",
                                 "Box-график распределения по выздоровевшим(в день) за неделю:"))
#subplot_titles=("Статистика по новым заболевшим за последние 2 недели","Статистика по выздоровевшим за последние 2 недели")

for i in cols:
    fig.add_trace(
    go.Box(x=new_df1[i],name = i,boxmean = True,marker_color = color_dict.get(i)),row=1,col=1)

for i in cols:
    fig.add_trace(
    go.Box(x=new_df2[i],name = i,boxmean=True,marker_color = color_dict.get(i)),row=2,col=1)


fig.update_layout(
#     annotations=[
#         dict(
#             x=0.5,
#             y=-0.15,
#             showarrow=False,
#             text="Ветикальная пунктирная линия - среднее, Вертикальная сплошная - медиана распределения",
#             xref="paper",
#             yref="paper"),
        
#                 ],
    
    
                  width = 990, height = 1200,
    
    
                 showlegend=False,template ='ggplot2'
)

#fig.update_layout(height=1000, width=800,showlegend=False)
fig.show()